In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)
#block_size is for the length of the integers 
block_size = 8
#batch_size is how many of them are running in parallel
batch_size = 4

cuda


In [2]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text=f.read()
#print(text[:200])
chars=sorted(set(text))
print(chars)
#tokenizer consist of encoder and decoder 
#encoder-convert each element in this array to an integer eg- maybe '\n' would be 0
print(len(chars))
vocab_size = len(chars)


['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [3]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s:[string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

#encoded=encode('hello')
#decoded=decode(encoded)
#print(decoded)
#so these are character level tokenizers which takes each character and coverts it to integer equivalent.

data = torch.tensor(encode(text),dtype=torch.long)
print(data[:100])

tensor([80, 44, 61, 58,  1, 40, 71, 68, 63, 58, 56, 73,  1, 31, 74, 73, 58, 67,
        55, 58, 71, 60,  1, 58, 26, 68, 68, 64,  1, 68, 59,  1, 28, 68, 71, 68,
        73, 61, 78,  1, 54, 67, 57,  1, 73, 61, 58,  1, 47, 62, 79, 54, 71, 57,
         1, 62, 67,  1, 39, 79,  0,  1,  1,  1,  1,  0, 44, 61, 62, 72,  1, 58,
        55, 68, 68, 64,  1, 62, 72,  1, 59, 68, 71,  1, 73, 61, 58,  1, 74, 72,
        58,  1, 68, 59,  1, 54, 67, 78, 68, 67])


In [4]:
#training and validation(Training from 80% percent of the data so our output will not be same as the whole input data i.e 100%)
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    # getting a random integer between zero and length of the entire text - block_size so if we get at the index at len(data)-block_size
    # we still will have space to get upto length of data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i+block_size] for i in ix])
    y = torch.stack([data[i+1 : i+1+block_size] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y
    
x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)
    
        


inputs:
tensor([[72,  1, 72, 74, 71, 58,  0, 73],
        [62, 72,  1, 76, 54, 78,  9,  1],
        [61, 62, 72,  1, 61, 54, 73,  1],
        [59, 54, 56, 58,  1, 54, 67, 57]], device='cuda:0')
targets:
tensor([[ 1, 72, 74, 71, 58,  0, 73, 61],
        [72,  1, 76, 54, 78,  9,  1, 69],
        [62, 72,  1, 61, 54, 73,  1, 73],
        [54, 56, 58,  1, 54, 67, 57,  1]], device='cuda:0')


In [5]:
#bigram-given a given input like a character predicts the next is bigram language model
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]

for i in range(block_size):
    context = x[:i+1]
    target = y[i]
    print("when input is" , context , "target is", target )
    


when input is tensor([80]) target is tensor(44)
when input is tensor([80, 44]) target is tensor(61)
when input is tensor([80, 44, 61]) target is tensor(58)
when input is tensor([80, 44, 61, 58]) target is tensor(1)
when input is tensor([80, 44, 61, 58,  1]) target is tensor(40)
when input is tensor([80, 44, 61, 58,  1, 40]) target is tensor(71)
when input is tensor([80, 44, 61, 58,  1, 40, 71]) target is tensor(68)
when input is tensor([80, 44, 61, 58,  1, 40, 71, 68]) target is tensor(63)


In [7]:
class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self,index,targets):
        #logits are bunch of floating points that are normalized. its contribution to the sum of everything
        #the context of the softmax function, "normalized" means that the output values are turned into probabilities.
        #Each output value will be between 0 and 1, and when you add up all the output values, they will equal 1.
        #eg-[2,4,6] and we got a totalof 12 now we got [2/12,4/12,6/12]=[0.167,0.33,0.5]
        logits = self.token_embedding_table(index)

                
        # If targets are not provided, set loss to None (useful during inference).
        if target is None:
            loss = None
            
        else:        
            # Unpack the shape of the logits tensor.
            #where B is the batch size, T is the sequence length (number of tokens in each sample), 
            #and C is the embedding size (dimensionality of the embedding vectors).
            B,T,C = logits.shape
            
            #view- allow us to unpack with dot shape and then we can use them to put them back in the tensor
            # Reshape logits and targets to match the expected input shape for cross-entropy loss.
            logits = logits.view(B*T, C)
            targets = targets.view(B*t)
            
            # Compute the cross-entropy loss between logits and targets.
            loss = F.cross_entropy(logits,targets)
        return logits,loss
    
    
    # This function generates a sequence of new tokens based on an initial sequence
    def generate(self,index,max_new_tokens):
        for i in range(max_new_tokens):
            # Get the logits for the current sequence using the forward pass
            logits,loss = self.foward(index)
            # Focus only on the logits for the last time step
            logits = logits[:,-1,:]
            # Convert the logits to probabilities using the softmax function
            #The dim=-1 argument in the probs = F.softmax(logits, dim=-1) line specifies that the softmax function should be 
            #applied along the last dimension of the logits tensor.
            prob = F.softmax(logits,dim = -1)
            # Sample a new token index from the probability distribution
            #introduces randomness into the generation process, allowing the model to generate diverse and plausible text. 
            #The sampled token index (index_next) is then used to extend the current sequence, and the process repeats to generate additional tokens.
            index_next = torch.multinomial(prob, num_sample = 1)
            # Append the new token index to the current sequence
            index = torch.cat((index, index_next), dim=1)
        return index


model = BigramLanguageModel(vocab_size)
    
# Create an initial context sequence with a single token (index 0)
context = torch.zeros((1,1), dtype=torch.long, device=device)
# Convert the generated token indices to characters (assuming a decode function exists)
# This step is for demonstration purposes and would depend on how the tokens are encoded
generated_chars = decode(generated_sequence[0].tolist())
    
# Print the generated sequence of characters
print(generated_chars)


        
        


        
    

NameError: name 'generated_sequence' is not defined